In [ ]:
#Amazon target
# 1_000_000_000 events per day
# ~11_500 events per second
# ~9.2 MB/s
# ~800 bytes per event message

#Install packages
!pip install neo4j-rust-ext
!pip install pandas
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.9/284.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 6.9 MB/s eta 0:00:00


In [ ]:
#Import packages and set env variables
import neo4j
from neo4j import GraphDatabase
import random
from threading import Thread
import threading
from tqdm import tqdm
from time import perf_counter
import time
import pandas
import requests
import base64
import statistics
import warnings
import sys
from types import ModuleType, FunctionType
from gc import get_referents

#32GB Aura Instance
NEO4J_USERNAME = "neo4j"
NEO4J_URI_5 = "neo4j+s://7a1a5e39.databases.neo4j.io"
NEO4J_PASSWORD_5 = "SVVAyymPa5jmojVH6-ddy0agBIxBYh0rqqHGJafmGqI"
NEO4J_AUTH_5 = (NEO4J_USERNAME, NEO4J_PASSWORD_5)
NEO4j_DATABASE = "neo4j"

class Neo4jInfo():
  def __init__(self, uri, auth, version, database):
    self.uri = uri
    self.auth = auth
    self.version = version
    self.database = database

neo4j_info = Neo4jInfo(NEO4J_URI_5, NEO4J_AUTH_5, 5, NEO4j_DATABASE)

ModuleNotFoundError: No module named 'neo4j'

In [ ]:
#Test connection
def can_connect(neo4j_info):
  with GraphDatabase.driver(neo4j_info.uri, auth=neo4j_info.auth) as driver:
    try:
      driver.verify_connectivity() #Throws if no connection
      return True
    except Exception as e:
      print(e)
      return False

def run_cypher(session, cypher, **kwargs):
  result = session.run(cypher, kwargs)
  values = []
  for record in result:
    values.append(record.values())
  summary = result.consume()
  return summary.counters

if can_connect(neo4j_info):
  print("Can connect to v5")
else:
  print("Failed to connect to v5")


Can connect to v5


In [ ]:
#Count and delete nodes, to reset database when required

count_nodes_cypher = """
MATCH (n{})
RETURN count(n)
"""

delete_all_cypher_5 = """
MATCH (n{})
CALL (n) {{
  DETACH DELETE n
}}
IN TRANSACTIONS OF 1000 ROWS
"""

def get_count_nodes_cypher(label_id = ""):
  if label_id != "":
    return count_nodes_cypher.format(":label_id_{}".format(label_id))
  else:
    return count_nodes_cypher.format("")

def get_delete_nodes_cypher(label_id = ""):
  if label_id != "":
    return delete_all_cypher_5.format(":label_id_{}".format(label_id))
  else:
    return delete_all_cypher_5.format("")

def count_nodes(session, label_id=""):
  cypher = get_count_nodes_cypher(label_id)
  result = session.run(cypher)
  count = 0
  for record in result:
    count += record.values()[0]
  summary = result.consume()
  return count

def delete_all_nodes(session, neo4j_info, label_id=""):
  cypher = get_delete_nodes_cypher(label_id)
  result = session.run(cypher)
  count = 0
  for record in result:
    count += record.values()[0]
  summary = result.consume()
  return count

def test_count_nodes(neo4j_info, label_id=""):
  with GraphDatabase.driver(neo4j_info.uri, auth=neo4j_info.auth) as driver:
    with driver.session(database=neo4j_info.database) as session:
      return count_nodes(session, label_id)

#Delete nodes
def test_delete_all_nodes(neo4j_info, label_id=""):
  with GraphDatabase.driver(neo4j_info.uri, auth=neo4j_info.auth) as driver:
    with driver.session(database="neo4j") as session:
      return delete_all_nodes(session, neo4j_info, label_id)

print("Counted {} nodes on v{}".format(test_count_nodes(neo4j_info), neo4j_info.version))
print("Deleted {} nodes on v{}".format(test_delete_all_nodes(neo4j_info), neo4j_info.version))
print("Counted {} nodes on v{}".format(test_count_nodes(neo4j_info), neo4j_info.version))

Counted 1500 nodes on v5
Deleted 0 nodes on v5
Counted 0 nodes on v5


In [ ]:
#Create/delete nodes as a source of changes to drive CDC events

create_nodes_cypher = """
UNWIND range(1,$node_count) AS i
CREATE (n:label_id_{label})
SET n.payload = $payload
"""

delete_nodes_cypher = """
MATCH (n:label_id_{label})
DETACH DELETE n
"""

def get_create_nodes_cypher(label_id):
  return create_nodes_cypher.format(label=label_id)

def get_delete_nodes_cypher(label_id):
  return delete_nodes_cypher.format(label=label_id)

def get_payload(payload_bytes):
  return bytes(payload_bytes)

def create_nodes(session, node_count, label_id, payload_bytes):
  cypher = get_create_nodes_cypher(label_id)
  payload = get_payload(payload_bytes)
  run_cypher(session, cypher, node_count=node_count, payload=payload)

def delete_nodes(session, label_id):
  cypher = get_delete_nodes_cypher(label_id)
  run_cypher(session, cypher)

def test_create_nodes(neo4j_info, node_count, label_id, payload_bytes):
  with GraphDatabase.driver(neo4j_info.uri, auth=neo4j_info.auth) as driver:
    with driver.session(database=neo4j_info.database) as session:
      create_nodes(session, node_count, label_id, payload_bytes)

def test_delete_nodes(neo4j_info, label_id):
  with GraphDatabase.driver(neo4j_info.uri, auth=neo4j_info.auth) as driver:
    with driver.session(database=neo4j_info.database) as session:
      delete_nodes(session, label_id)

label_id = "test"
print("Counted {} nodes with label {} on v{}".format(test_count_nodes(neo4j_info, label_id), label_id, neo4j_info.version))
test_create_nodes(neo4j_info, 100, label_id, 800)
print("Counted {} nodes with label {} on v{}".format(test_count_nodes(neo4j_info, label_id), label_id, neo4j_info.version))
test_delete_nodes(neo4j_info, label_id)
print("Counted {} nodes with label {} on v{}".format(test_count_nodes(neo4j_info, label_id), label_id, neo4j_info.version))

Counted 0 nodes with label test on v5
Counted 100 nodes with label test on v5
Counted 0 nodes with label test on v5


In [ ]:
#Class that continuously checks CDC in background thread

retrieve_changes_cypher = """
CALL db.cdc.query($previous_id)
YIELD id, txId, seq, metadata, event
RETURN id, txId, seq, metadata, event
LIMIT 10000
"""

get_current_id_cypher = """
CALL db.cdc.current() YIELD id RETURN id
"""

import sys

class cdc:

  def __init__(self):
    self.neo4j_info = None
    self.driver = None
    self.session = None
    self.previous_id = None
    self.running = False

  def __enter__(self):
    return self

  def __exit__(self, exc_type, exc_value, traceback):
    self.close()

  def open(self, neo4j_info):
    self.close()
    self.neo4j_info = neo4j_info
    self.driver = GraphDatabase.driver(neo4j_info.uri, auth=neo4j_info.auth)
    self.session = self.driver.session(database=neo4j_info.database)
    self.previous_id = None
    self.first_event_size = None

  def close(self):
    if self.running: self.wait_stop()
    if self.session is not None:
      self.session.close()
      self.session = None
    if self.driver is not None:
      self.driver.close()
      self.driver = None

  def start(self):
    if self.running: return

    self.running = True
    self.start_time = time.time()
    self.total_captured = 0

    self.thread = Thread(target=self.cdc_thread, args=(self.driver, lambda : self.running))
    self.thread.start()

  def send_stop(self):
    self.running = False
    self.stop_thread = Thread(target=self.time_stop_thread, args=())
    self.stop_thread.start()

  def time_stop_thread(self):
    self.thread.join()
    self.stop_time = time.time()
    self.total_runtime = self.stop_time - self.start_time

  def wait_stop(self):
    self.send_stop()
    self.stop_thread.join()
    return self.total_captured, self.total_runtime, self.first_event_size

  def cdc_thread(self, driver, running):
    with driver.session(database="neo4j") as session:
      while running():
        self.total_captured += self.retrieve_changes(session)
        time.sleep(0.001)

  def reset_id_to_current(self):
    result = self.session.run(get_current_id_cypher)
    return result.single()[0]

  def retrieve_changes(self, session):
    if self.previous_id == None:
      self.previous_id = self.reset_id_to_current()
    result = session.run(retrieve_changes_cypher, previous_id=self.previous_id)
    changes = []
    count = 0
    for record in result:
      count += 1
      self.previous_id = record.values()[0]
      if self.first_event_size == None:
        self.first_event_size = sys.getsizeof(str(record))
    return count

cdc = cdc()

def test_cdc():
  label_id = "test_1"
  test_delete_nodes(neo4j_info, label_id)
  print("Counted {} nodes with label {} on v{}".format(test_count_nodes(neo4j_info, label_id), label_id, neo4j_info.version))

  cdc.open(neo4j_info)
  cdc.start()
  test_create_nodes(neo4j_info, 100, label_id, 1)
  print("Counted {} nodes with label {} on v{}".format(test_count_nodes(neo4j_info, label_id), label_id, neo4j_info.version))
  count = cdc.total_captured
  print("Retrieved {} event changes".format(count))
  print("Deleting Nodes...")
  test_delete_nodes(neo4j_info, label_id)
  print("Counted {} nodes with label {} on v{}".format(test_count_nodes(neo4j_info, label_id), label_id, neo4j_info.version))
  time.sleep(1)
  count = cdc.total_captured
  print("Retrieved {} event changes".format(count))
  total_captured, total_time, first_event_size = cdc.wait_stop()
  cdc.close()
  print("Total time: {}".format(total_time))
  print("Total captured: {}".format(total_captured))
  print("First event size: {}".format(first_event_size))

test_cdc()

Counted 0 nodes with label test_1 on v5
Counted 100 nodes with label test_1 on v5
Retrieved 100 event changes
Deleting Nodes...
Counted 0 nodes with label test_1 on v5
Retrieved 200 event changes
Total time: 5.38709831237793
Total captured: 200
First event size: 825


In [ ]:
#Single-threaded stress test of CDC

changed_count = 0
label_id = "test_3"
cdc.open(neo4j_info)
with GraphDatabase.driver(neo4j_info.uri, auth=neo4j_info.auth) as driver:
  with driver.session(database=neo4j_info.database) as session:
    start = time.time()
    cdc.start()
    while(time.time() - start < 10):
      create_nodes(session, 50, label_id, 1)
      changed_count += 50
      delete_nodes(session, label_id)
      changed_count += 50
      time.sleep(1)
    cdc.send_stop()

total_captured, total_time, first_event_size = cdc.wait_stop()
cdc.close()

print("Change count {}, Captured count {}".format(changed_count, total_captured))

Change count 800, Captured count 800


In [ ]:
#Class that continuously checks create database changes in background thread

import multiprocessing
import math

class change_maker:

  def __init__(self):
    self.max_threads = multiprocessing.cpu_count() * 2
    if self.max_threads > 1:
      self.max_threads -= 1
    self.max_batch_size = 3_000
    self.running = False
    self.session = None
    self.driver = None

  def __enter__(self):
    return self

  def __exit__(self, exc_type, exc_value, traceback):
    self.close()

  def open(self, neo4j_info, target_change_rate=1, payload_bytes=1):
    self.close()
    self.lock = threading.Lock()
    self.neo4j_info = neo4j_info
    self.driver = GraphDatabase.driver(neo4j_info.uri, auth=neo4j_info.auth)
    self.session = self.driver.session(database=neo4j_info.database)
    self.target_change_rate = target_change_rate
    self.target_change_rate_per_thread = -1
    if self.target_change_rate > 0:
      self.target_change_rate_per_thread = target_change_rate / self.max_threads
    self.payload_bytes = payload_bytes

  def close(self):
    if self.running: self.wait_stop()
    if self.session is not None:
      self.session.close()
      self.session = None
    if self.driver is not None:
      self.driver.close()
      self.driver = None

  def start(self):
    if self.running: return

    self.running = True
    self.start_time = time.time()
    self.total_changes = 0

    self.threads = []
    for thread_id in range(self.max_threads):
      thread = Thread(target=self.change_maker_thread, args=(self.driver, thread_id, self.target_change_rate_per_thread, lambda : self.running))
      self.threads.append(thread)
    for thread in self.threads:
      thread.start()

  def send_stop(self):
    self.stop_thread = Thread(target=self.time_stop_thread, args=())
    self.stop_thread.start()

  def time_stop_thread(self):
    self.running = False
    for thread in self.threads:
      thread.join()
    self.stop_time = time.time()
    self.total_runtime = self.stop_time - self.start_time

  def wait_stop(self):
    self.send_stop()
    self.stop_thread.join()
    return self.total_changes, self.total_runtime

  def change_maker_thread(self, driver, thread_id, target_change_rate_per_thread, running):
    my_start_time = perf_counter()
    my_total_changes = 0
    my_batch_size = self.max_batch_size
    if target_change_rate_per_thread > 0:
      my_batch_size = math.ceil(min(target_change_rate_per_thread/2, my_batch_size))

    with driver.session(database="neo4j") as session:
      while running():

        create_nodes(session, my_batch_size, thread_id, payload_bytes=self.payload_bytes)
        my_total_changes += my_batch_size
        with self.lock:
          self.total_changes += my_batch_size
        self.wait_if_necessary(my_start_time, my_total_changes, target_change_rate_per_thread, thread_id)

        if not running(): break
        time.sleep(0.001)

        delete_nodes(session, thread_id)
        my_total_changes += my_batch_size
        with self.lock:
          self.total_changes += my_batch_size
        self.wait_if_necessary(my_start_time, my_total_changes, target_change_rate_per_thread, thread_id)
        time.sleep(0.001)

  def wait_if_necessary(self, my_start_time, my_total_changes, target_change_rate_per_thread, thread_id):
    if target_change_rate_per_thread <= 0:
      return

    elapsed_time = perf_counter() - my_start_time
    expected_changes = elapsed_time * target_change_rate_per_thread

    #if thread_id == 0:
    #  print("elapse: {}, actual changes {}, expected changes {}".format(elapsed_time, my_total_changes, expected_changes))

    excess_changes = my_total_changes - expected_changes
    if excess_changes <= 0:
      return

    required_delay = excess_changes / target_change_rate_per_thread
    time.sleep(required_delay)

cm = change_maker()

def test_make_changes(neo4j_info, target_change_rate, test_duration):
  cm.open(neo4j_info, target_change_rate=target_change_rate)
  cm.start()
  time.sleep(test_duration)
  total_changes, total_runtime = cm.wait_stop()
  cm.close()
  return total_changes, total_runtime

total_changes, total_runtime = test_make_changes(neo4j_info, 50_000, 10)
print("Actual change rate: {:,.0f}".format(total_changes/total_runtime))
print("Total time: {}".format(total_runtime))
print("Total changes: {}".format(total_changes))

Actual change rate: 49,989
Total time: 10.082184314727783
Total changes: 504000


In [ ]:
#Test change_make's ability to hit specific change rates

target_rate = 1000
increment = 1000
test_duration = 10

while target_rate <= 5_000:
  total_changes, total_runtime = test_make_changes(neo4j_info, target_rate, test_duration)
  print("Target rate: {:,.0f}, Actual change rate: {:,.0f}, Target time: {:,.1f}, Actual time: {:,.1f}".format(target_rate, total_changes/total_runtime, test_duration, total_runtime))
  target_rate += increment

Target rate: 1,000, Actual change rate: 1,000, Target time: 10.0, Actual time: 10.0
Target rate: 2,000, Actual change rate: 1,999, Target time: 10.0, Actual time: 10.0
Target rate: 3,000, Actual change rate: 2,999, Target time: 10.0, Actual time: 10.5
Target rate: 4,000, Actual change rate: 4,000, Target time: 10.0, Actual time: 10.5
Target rate: 5,000, Actual change rate: 4,998, Target time: 10.0, Actual time: 10.0


In [ ]:
#Individual test of CDC's ability to keep up with change_maker

def performance_test(target_change_rate, test_time, payload_bytes=1):
  test_delete_all_nodes(neo4j_info)
  time.sleep(1)
  cdc.open(neo4j_info)
  cm.open(neo4j_info, target_change_rate=target_change_rate, payload_bytes=payload_bytes)
  cdc.start()
  cm.start()
  time.sleep(test_time)
  cm.send_stop()
  cdc.send_stop()
  time.sleep(0.001)
  total_captured, total_cdc_time, first_event_size = cdc.wait_stop()
  total_changes, total_cm_time = cm.wait_stop()
  cm.close()
  cdc.close()

  print("{:,.0f}\t{:,.0f}\t{:,.0f}\t{:,.0f}\t{:,.1f}\t{:,.0f}\t{:,.1f}".format(first_event_size, target_change_rate, test_time, total_changes, total_cm_time, total_captured, total_cdc_time))
  return total_changes, total_cm_time, total_captured, total_cdc_time, first_event_size

target_change_rate = 3000
test_time = 10 #seconds
payload_bytes = 1
total_changes, total_cm_time, total_captured, total_cdc_time, first_event_size = performance_test(target_change_rate, test_time, payload_bytes)

print("Target Rate: {:,.0f}, Test Time: {:,.0f}".format(target_change_rate, test_time))
print("Total Changes: {:,.0f}, Total Time: {:,.1f}s, Change Rate: {:,.0f}".format(total_changes, total_cm_time, total_changes/total_cm_time))
print("Total Events: {:,.0f}, Total Time: {:,.1f}s, Event Rate: {:,.0f}".format(total_captured, total_cdc_time, total_captured/total_cdc_time))
print("First event size: {:,.0f} bytes".format(first_event_size))


817	3,000	10	31,500	10.6	25,500	10.6
Target Rate: 3,000, Test Time: 10
Total Changes: 31,500, Total Time: 10.6s, Change Rate: 2,985
Total Events: 25,500, Total Time: 10.6s, Event Rate: 2,416
First event size: 817 bytes


In [ ]:
#Automate tests of CDC's ability to keep up with change_maker
#Example results: https://docs.google.com/spreadsheets/d/1mc33GeNxbMYku36JZ68ieWV4RgMhMfy0y2tt8cpQdA0/edit?gid=619424928#gid=619424928

rate = 250
increment = 250
test_time = 10
while rate <= 5_000:
  performance_test(rate, test_time)
  rate += increment

813	250	10	2,520	10.1	5,022	10.0
814	500	10	5,040	10.1	5,040	10.0
812	750	10	7,875	10.5	7,500	10.1
813	1,000	10	10,020	10.0	10,395	10.0
814	1,250	10	12,540	10.1	12,540	10.1
811	1,500	10	15,750	10.5	15,000	10.1
813	1,750	10	17,520	10.1	18,270	10.1
811	2,000	10	20,040	10.6	20,040	10.6
813	2,250	10	23,625	10.5	21,375	10.3
813	2,500	10	25,854	10.6	21,251	10.6
814	2,750	10	27,540	10.5	21,377	10.5
814	3,000	10	31,500	10.6	21,500	10.6
812	3,250	10	34,146	10.7	21,626	10.7
814	3,500	10	35,040	10.6	21,752	10.6
812	3,750	10	39,375	11.3	21,875	11.3
812	4,000	10	42,021	11.2	22,001	11.2
814	4,250	10	43,249	11.3	22,127	11.3
812	4,500	10	47,250	11.4	22,250	11.4
813	4,750	10	49,896	11.5	22,376	11.5
813	5,000	10	52,542	11.4	22,502	11.4
